# Case Study 1 : Collecting Data from Twitter

Due Date: September 22, **before the beginning of class at 6:00pm**

* ------------

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Twitter_bird_logo_2012.svg/220px-Twitter_bird_logo_2012.svg.png">

**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

Guanxiong Liu
Jiankun Bi
Abhishek Easwaran
Naveen Pothayath
Suchithra Balakrishnan    

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](http://www.learndatasci.com/wp-content/uploads/2015/08/Mining-the-Social-Web-2nd-Edition.pdf) 
* The codes for [Chapter 1](http://bit.ly/1qCtMrr) and [Chapter 9](http://bit.ly/1u7eP33)


** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

# Problem 1: Sampling Twitter Data with Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [1]:
import twitter
import time
from collections import Counter
from prettytable import PrettyTable

#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://dev.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'iEG3V3LLXNerRncmEWgv8lQ4n'
    CONSUMER_SECRET ='67HGDVQRF7YxJ6ZUFnUlvwreR2CShhao1Zx6k5wo0OoRRkGez4'
    OAUTH_TOKEN = '2380166015-sZywqhm3U9naQ154u0qQQzBv1FJgAWDVSWN6ROB'
    OAUTH_TOKEN_SECRET = '1AQCSvqp06DTpwYbgBXooWVmLuzWJ8zpsFpWgaazUhhha'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
import json

# login to api
twitter_api = oauth_login()

# set initial search
search_results = twitter_api.search.tweets(q='driverless', count=100, lang='en')
statuses = search_results['statuses']
print("Length of statuses", len(statuses))
max_tweets = 1000

while(len(statuses) < max_tweets):
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError: # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
    
    # search api will take the key words dictionary
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']
    print("Length of statuses", len(statuses))

# Write the status to a local file
outfile = open('tweets.txt', 'w')
outfile.write(json.dumps(statuses, indent=1))
outfile.close()
print ("Tweets output to file successfully!")









Length of statuses 100
Length of statuses 200
Length of statuses 300
Length of statuses 400
Length of statuses 500
Length of statuses 589
Tweets output to file successfully!


### Report some statistics about the tweets you collected 

* The topic of interest: < INSERT YOUR TOPIC HERE>


* The total number of tweets collected:  < INSERT THE NUMBER HERE>

*-----------------------

# Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [3]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Collect status texts
from nltk.corpus import stopwords
status_texts = [ status['text'] 
                 for status in statuses ]

# Use word to store all words in tweets and use counter to do analysis
word = []
for text in status_texts:
    for w in text.split():
        word.append(w)
filter_words = [word for word in word if word not in stopwords.words('english')]
word_freq = Counter(filter_words)

# Put the data into the prettytable
top_word = PrettyTable(field_names=['Word','Count']) # Header
row_num = 0 # number of rows in table
for item in word_freq.most_common()[:20]:
    top_word.add_row(item) # Adding rows
top_word.align['Word'], top_word.align['Count'] = 'c', 'r' # Set column alignment
print (top_word)




+------------+-------+
|    Word    | Count |
+------------+-------+
| driverless |   324 |
|     RT     |   241 |
|    cars    |   138 |
|    car     |   107 |
|    cost    |   105 |
|    per     |    86 |
| Driverless |    86 |
|    mile    |    86 |
|     A      |    78 |
|    The     |    56 |
|    take    |    53 |
|     -      |    46 |
|   Could    |    45 |
|    taxi    |    44 |
|   taxi:    |    43 |
| Estimated  |    43 |
|     35     |    43 |
|   $3.46    |    43 |
|   cents    |    43 |
|  Average   |    43 |
+------------+-------+


**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [4]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Collect all retweets
retweets = []
for status in statuses:
    if 'retweeted_status' in status:
        retweets.append((status['retweet_count'], status['retweeted_status']['user']['screen_name'], status['text']))

# Generate the pretty table
most_retweet = PrettyTable(field_names=['Count', 'Screen Name', 'Tweet Text']) # Header
for item in sorted(set(retweets), reverse=True)[:10]:
        most_retweet.add_row(item) # Adding rows
most_retweet.align['Count'], most_retweet.align['Screen Name'], most_retweet.align['Tweet Text'] = 'l', 'l', 'l'
most_retweet.max_width['Tweet Text'] = 50 # Set max width for "Tweet Text" column
print (most_retweet)











+-------+-----------------+----------------------------------------------------+
| Count | Screen Name     | Tweet Text                                         |
+-------+-----------------+----------------------------------------------------+
| 790   | ericries        | RT @ericries: I can't believe we call them         |
|       |                 | driverless cars, knowing they were once called     |
|       |                 | horseless carriages.                               |
| 446   | WPXI            | RT @WPXI: NOW ON 11: Uber to debut driverless cars |
|       |                 | in #Pittsburgh starting today #WPXI                |
|       |                 | https://t.co/H27v6cSgRQ https://t.co/30WkNGvXA3    |
| 382   | weknowwhatsbest | RT @weknowwhatsbest: Google scientists invented    |
|       |                 | the driverless car and got the idea after watching |
|       |                 | the Obama presidency.                              |
| 342   | MarketWatch     | 

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [10]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Collect hashtags
hashtags = [ hashtag['text'] 
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

# Collect mention user
mention_user = [ user_mention['screen_name'] 
                 for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]

# Generate pretty table
for label, data in (('Hashtag',hashtags), ('Mention User', mention_user)): # Access two table data
    table = PrettyTable(field_names=[label, 'Count']) # Header
    count = Counter(data) # Count the data
    for item in count.most_common()[:10]:
        table.add_row(item) # Adding each row
    table.align[label], table.align['Count'] = 'l', 'c'
    print (table)






+------------+-------+
| Hashtag    | Count |
+------------+-------+
| driverless |   21  |
| tech       |   8   |
| Uber       |   6   |
| Lyft       |   5   |
| AI         |   5   |
| Driverless |   4   |
| business   |   4   |
| Seattle    |   3   |
| Boston     |   3   |
| Tech       |   3   |
+------------+-------+
+-----------------+-------+
| Mention User    | Count |
+-----------------+-------+
| businessinsider |   17  |
| Uber            |   7   |
| JMBooyah        |   6   |
| SAI             |   5   |
| FerRomero_FREE  |   5   |
| evankirstel     |   4   |
| Laraba811       |   4   |
| markets         |   4   |
| DigitalTrends   |   3   |
| techradar       |   3   |
+-----------------+-------+


* ------------------------

# Problem 3: Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [5]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Get all friends
friends = twitter_api.friends.ids(screen_name='AutonomousCaRR', count=5000)
friend_list = friends['ids']
next_cursor = friends['next_cursor']
while (next_cursor != 0 and len(friend_list) < 5000*15): # Navigate through pages by cursor and rate controller
    friends = twitter_api.friends.ids(screen_name='AutonomousCaRR', cursor=next_cursor, count=5000)
    friend_list += friends['ids']
    next_cursor = friends['next_cursor']


# Get all followers
followers = twitter_api.followers.ids(screen_name='AutonomousCaRR', count=5000)
follower_list = followers['ids']
next_cursor = followers['next_cursor']
rate_counter = 1
while (next_cursor != 0 and len(follower_list) < 5000*15): # Navigate through pages by cursor and rate controller
    followers = twitter_api.followers.ids(screen_name='AutonomousCaRR', cursor=next_cursor, count=5000)
    follower_list += followers['ids']
    next_cursor = followers['next_cursor']




In [6]:
# Create table for friend
friend_table = PrettyTable(field_names=['Friend ID', 'Screen Name']) # Header
for user in twitter_api.users.lookup(user_id=friend_list[:20]):
    friend_table.add_row([user['id'], user['screen_name']])
friend_table.align['Friend ID'], friend_table.align['Screen Name'] = 'l', 'c' # Alignment
print (friend_table)

# Create table for follower
follower_table = PrettyTable(field_names=['Follower ID', 'Screen Name']) # Header
for user in twitter_api.users.lookup(user_id=follower_list[:20]):
    follower_table.add_row([user['id'], user['screen_name']])
follower_table.align['Follower ID'], follower_table.align['Screen Name'] = 'l', 'c' # Alignment
print (follower_table)

+------------+-----------------+
| Friend ID  |   Screen Name   |
+------------+-----------------+
| 40796368   |     kaybed1     |
| 223514649  |   Zeyawinhtut   |
| 292825731  |     smkayes     |
| 113802609  |  RachaelAnne_3  |
| 104746157  |    Sorafatul    |
| 235160347  |     rbenlee     |
| 1669662031 | saikatsamanta18 |
| 52056156   |    Mwardi300    |
| 465888120  |     HP101201    |
| 473720203  |   ferryantonie  |
| 159912065  |  DataRecoverer  |
| 411747568  |  heavydutyrigs  |
| 136652675  |     mwahyyd     |
| 1094487468 |  NichakornBogy  |
| 419777782  |  alamashraf2020 |
| 302055316  |     anjan_91    |
| 111618328  |  UMANGLOVESLIFE |
| 275031863  |  JenBDesjardins |
+------------+-----------------+
+--------------------+----------------+
| Follower ID        |  Screen Name   |
+--------------------+----------------+
| 777961129649680386 | removingsteps  |
| 400646160          |  follow_eiver  |
| 108813774          |  AutomotiveUX  |
| 1940136116         |    kt3_mame

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [7]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Create Set for friend and follower
friend = set(friend_list)
follower = set(follower_list)

# Check mutual friend
mutual_friend = friend.intersection(follower)
mutual_friend = list(mutual_friend)
if (len(mutual_friend) == 0):
    print ("There are no mutual friends.")
else:
    mutual_table = PrettyTable(field_names=['Mutual Friend ID', 'Screen Name']) # Create table for mutual friend
    for mutual in twitter_api.users.lookup(user_id=mutual_friend[:30]):
        mutual_table.add_row([mutual['id'], mutual['screen_name']])
    mutual_table.align['Mutual Friend ID'], mutual_table.align['Screen Name'] = 'l', 'c'
    print (mutual_table)






+------------------+-----------------+
| Mutual Friend ID |   Screen Name   |
+------------------+-----------------+
| 857645059        |  AutomotivePRBE |
| 1256669196       |    wiemsoukeh   |
| 998318094        |      tc604      |
| 2338316309       |   FloreaMatei   |
| 2849882135       |  XclusiveAutoNJ |
| 2692718616       | MrDWilliamsBGHS |
| 353665049        |      Biltac     |
| 21495840         |     gr8n8no     |
| 19578913         |      Verio      |
| 2373877795       |  Ancaster_Group |
| 256655396        |   orilayeelay   |
| 35545125         |    FresnoBill   |
| 41295909         |     redsox54    |
| 54575144         |    carlisence   |
| 378019885        |    KevTynan10   |
| 2754117678       |    avtomaser    |
| 281337902        |     Suren_F1    |
| 204955698        |   jacobyaaron   |
| 48906292         |     c230mike    |
| 546054204        |  AndreaRapelli  |
| 237846600        |  Ken_Simmons_NL |
| 214237258        |    xaviroca1    |
| 268247115        |  Tre

*------------------------

# Problem 4: Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [1]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

import twitter
import json
import pymongo
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://dev.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'iEG3V3LLXNerRncmEWgv8lQ4n'
    CONSUMER_SECRET ='67HGDVQRF7YxJ6ZUFnUlvwreR2CShhao1Zx6k5wo0OoRRkGez4'
    OAUTH_TOKEN = '2380166015-sZywqhm3U9naQ154u0qQQzBv1FJgAWDVSWN6ROB'
    OAUTH_TOKEN_SECRET = '1AQCSvqp06DTpwYbgBXooWVmLuzWJ8zpsFpWgaazUhhha'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

def mongoDB_connect():
    client = pymongo.MongoClient()
    return client




In [2]:
# login to api
twitter_api = oauth_login()
# set up database
client = mongoDB_connect()
db = client.ds501case1
collection = db.tweets

In [97]:
# main process to connect database and collect new data
def collecte_DB(max_tweets, key_word, clean_history):
    # parameter initial
    index = 0
    statuses = []
    
    # clean history data
    if clean_history:
        collection.delete_many({})
        print 'History Data are cleaned'

    for index in range(len(key_word)):
        # initial search
        word = key_word[index]
        search_results = twitter_api.search.tweets(q=word, count=100, lang='en')
        statuses += search_results['statuses']

        # continue get result
        while(len(statuses) < (max_tweets*(index+1))):
            try:
                next_results = search_results['search_metadata']['next_results']
            except KeyError: # No more results when next_results doesn't exist
                break

            # Create a dictionary from next_results, which has the following form:
            # ?max_id=313519052523986943&q=NCAA&include_entities=1
            kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])

            # search api will take the key words dictionary
            search_results = twitter_api.search.tweets(**kwargs)
            statuses += search_results['statuses']
    print "Final volume of problem 4 tweets is %d" % len(statuses)

    # write data into mongoDB
    for status in statuses:
        res = collection.insert_one(status)
    print "Problem 4 database is generated successfully!"

    
# set initial parameters
max_tweets = 1000
key_word = ['driverless',
            '‪AutonomousCaRR‪',
            'googlecar',
            'autonomouscars',
            'selfdriving',
            'futuristiccars',
            'driverlesstechnology',
            'SelfDrivingUber',
            'UGV']

# collect data and write into DB, enable this line iff new tweet data is necessary
#collecte_DB(max_tweets, key_word, True)



History Data are cleaned
Final volume of problem 4 tweets is 5754
Problem 4 database is generated successfully!


## Sematic Orientation

In [3]:
import pandas as pd
import nltk
import re
import string
from collections import defaultdict
from collections import Counter
import numpy as np



# load words pool from Bing Liu
pos_pool = pd.read_table('positive-words.txt', encoding='ISO-8859-1')
neg_pool = pd.read_table('negative-words.txt', encoding='ISO-8859-1')
pos_pool.columns = ['word']
neg_pool.columns = ['word']
pos_pool = pos_pool['word'].tolist() 
neg_pool = neg_pool['word'].tolist()



# function to extract words inside the tweet
def extract_word(tweet):
    tweet = re.sub(r"http\S+", "", tweet)
    words = [e.lower() for e in nltk.tokenize.word_tokenize(tweet) 
             if ((len(e) >= 3) and (e not in stop) and (not e.startswith(('#','@'))))]
    return (words)



# build word counter, co-occurance matrix
com = defaultdict(lambda: defaultdict(int))
words_pool = []
stop = nltk.corpus.stopwords.words('english') + list(string.punctuation) + ['RT', 'rt', 'via']

for tweet in collection.find({}):
    text = extract_word(tweet['text']) # extract words
    
    words_pool += text # build words pool
    
    for i in range(len(text)-1): # build co-occurance matrix
        for j in range(i+1, len(text)):
            w1, w2 = sorted([text[i], text[j]])
            if w1 != w2:
                com[w1][w2] += 1

words_counter = Counter(words_pool)



# compute word prob
p_w = {}
p_com = defaultdict(lambda : defaultdict(int))
n_pool = float(len(words_pool))
 
for word, n in words_counter.items():
    p_w[word] = n / n_pool
    for cow in com[word]:
        p_com[word][cow] = com[word][cow] / n_pool

        
               
# compute pmi and semantic orientation
pmi = defaultdict(lambda : defaultdict(int))
for w1 in p_w:
    for w2 in com[w1]:
        denom = p_w[w1] * p_w[w2]
        pmi[w1][w2] = np.log2(p_com[w1][w2] / denom)

SO_m = {}
for word, n in p_w.items():
    positive_assoc = sum(pmi[word][tx] for tx in pos_pool)
    negative_assoc = sum(pmi[word][tx] for tx in neg_pool)
    SO_m[word] = positive_assoc - negative_assoc


In [47]:
import operator
from prettytable import PrettyTable


comp_name = ['google',
             'tesla',
             'uber',
             'apple',
             'ford'
             ]

for name in comp_name:
    pmi_val = pmi[name]
    pmi_val = sorted(pmi_val.items(),
                     key=operator.itemgetter(1),
                     reverse=True)
    title = 'The top 20 co-occurant word with ' + name
    print (title)
    comp_table = PrettyTable(field_names=['Word', 'PMI Value'])
    for item in pmi_val[:20]:
        comp_table.add_row([item[0],item[1]])
    print (comp_table)
    
    data_frame = pd.DataFrame(pmi_val)
    data_frame.columns = ['Word', 'PMI Value']
    file_name = name + '.csv'
    data_frame.to_csv(path_or_buf=file_name, header=True, index=False, encoding='UTF-8')



The top 20 co-occurant word with google
+---------------------+---------------+
|         Word        |   PMI Value   |
+---------------------+---------------+
|      techshare      |  8.8468042303 |
|         rnib        |  8.8468042303 |
|         neat        |  8.8468042303 |
|      googlemaps     |  8.8468042303 |
|       surveys       |  8.8468042303 |
| googleautonomouscar | 8.58376982446 |
|        indeed       | 8.26184172957 |
|        loses        | 7.98430775405 |
|       patents       | 7.90569791935 |
|        hires        |  7.8468042303 |
|       respond       |  7.8468042303 |
|    selfdrivngcar    |  7.8468042303 |
|      googlebike     |  7.8468042303 |
|        solved       |  7.8468042303 |
|        “they        |  7.8468042303 |
|        hinges       |  7.8468042303 |
|        roger        |  7.8468042303 |
|        tight        |  7.8468042303 |
|       mbrainfr      |  7.8468042303 |
|       technica      |  7.8468042303 |
+---------------------+---------------+


In [22]:
# explore the top positive and negative words
semantic_sorted = sorted(SO_m.items(), 
                         key=operator.itemgetter(1), 
                         reverse=True)
top_pos = semantic_sorted[:10]
top_neg = semantic_sorted[-10:]
top_neg.reverse()


pos_table = PrettyTable(field_names=['Word','Semantic Orientation Value'])
neg_table = PrettyTable(field_names=['Word','Semantic Orientation Value'])

for pos,neg in zip(top_pos,top_neg):
    pos_table.add_row([pos[0],pos[1]])
    neg_table.add_row([neg[0],neg[1]])


print(pos_table)
print(neg_table)

+------------+----------------------------+
|    Word    | Semantic Orientation Value |
+------------+----------------------------+
| autonomous |       102.595997287        |
|    day     |       74.5923936053        |
|    auto    |       67.1979537487        |
|    car     |       63.0891322622        |
|    amp     |       61.3199661459        |
|    ford    |       58.5628261828        |
|   a-ugv    |       58.1485382108        |
|   cross    |       53.6277545396        |
|   autos    |       52.4839898106        |
|   going    |       51.8391172464        |
+------------+----------------------------+
+------------+----------------------------+
|    Word    | Semantic Orientation Value |
+------------+----------------------------+
|   apple    |       -56.1334657501       |
|   could    |       -55.9001934622       |
| insurance  |       -52.5995170403       |
|    bad     |       -46.2039325324       |
|  freedom   |       -46.0580240552       |
|    also    |       -45.6672164

In [30]:
semantic_orientation = pd.DataFrame(semantic_sorted)
semantic_orientation.columns = ['Word','Semantic Orientation Value']
semantic_orientation.head()
semantic_orientation.to_csv('Semantic_Orientation.csv', header=True, index=False, encoding='UTF-8')

## Word Cloud

In [1]:
#creation of word cloud of negative words
import pandas as pd
import numpy as np
import matplotlib

values = pd.read_csv('Semantic_Orientation.csv')

In [2]:
valuess=values.Value
wordss=values.Word

i=0
vallist=[]
wordlist=[]
for words, values in zip(wordss, valuess):
    val=float(valuess[i])
    i=i+1
    if val<0:
        vallist.append(-val)
        wordlist.append(wordss[i-1])

In [3]:
import random,os

from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

import numpy as np
from wordcloud import query_integral_image

# FONT_PATH = "C:/Python33/Lib/site-packages/matplotlib/mpl-data/fonts/ttf/vera.ttf"
FONT_PATH = "C:/Users/Suchithra Bala/Anaconda3/Lib/site-packages/matplotlib/mpl-data/fonts/ttf/Vera.ttf"


def make_wordcloud(wordlist, vallist, fname=None, font_path=None, width=400, height=200,
                   margin=5, ranks_only=False, backgroundweight=255):
   
    if len(vallist) <= 0:
        print("We need at least 1 word to plot a word cloud, got %d."
            % len(vallist))

    if font_path is None:
        font_path = FONT_PATH

    if not os.path.exists(font_path):
        raise ValueError("The provided font %s does not exist." % font_path)

    # normalize value
    vallist=[float(i/max(vallist)) for i in vallist]
    # sort words by values
    inds = np.argsort(vallist)[::-1]
    vallist = [vallist[i] for i in inds]
    wordlist = [wordlist[i] for i in inds]
    # create image
    img_grey = Image.new("L", (width, height))
    draw = ImageDraw.Draw(img_grey)
    integral = np.zeros((height, width), dtype=np.uint32)
    img_array = np.asarray(img_grey)
    font_sizes, positions, orientations = [], [], []
    # intitiallize font size "large enough"
    font_size = 1000
    
    # start drawing grey image
    for word, value in zip(wordlist, vallist):
        # alternative way to set the font size
     
        if not ranks_only:
            font_size = min(font_size, int(100 * np.log(value + 100)))
        while True:
            # try to find a position
            font = ImageFont.truetype(font_path, font_size, encoding = 'unic')
            # transpose font optionally
            orientation = random.choice([None, Image.ROTATE_90])
            transposed_font = ImageFont.TransposedFont(font, orientation=orientation)
            draw.setfont(transposed_font)
            # get size of resulting text
            box_size = draw.textsize(word)
            # find possible places using integral image:
            result = query_integral_image.query_integral_image(integral, box_size[1] + margin,
                                          box_size[0] + margin, np.random.RandomState(seed=0))
            if result is not None or font_size == 0:
                break
            # if we didn't find a place, make font smaller
            font_size -= 1

        if font_size == 0:
            # we were unable to draw any more
            break

        x, y = np.array(result) + margin // 2
        # actually draw the text
        draw.text((y, x), word, fill="white")
        positions.append((x, y))
        orientations.append(orientation)
        font_sizes.append(font_size)
        # recompute integral image
        img_array = np.asarray(img_grey)
        # recompute bottom right
        # the order of the cumsum's is important for speed ?!
        partial_integral = np.cumsum(np.cumsum(img_array[x:, y:], axis=1),
                                     axis=0)
        # paste recomputed part into old image
        # if x or y is zero it is a bit annoying
        if x > 0:
            if y > 0:
                partial_integral += (integral[x - 1, y:]
                                     - integral[x - 1, y - 1])
            else:
                partial_integral += integral[x - 1, y:]
        if y > 0:
            partial_integral += integral[x:, y - 1][:, np.newaxis]

        integral[x:, y:] = partial_integral

    # redraw in color
    img = Image.new("RGB", (width, height), (backgroundweight,backgroundweight,backgroundweight))
    draw = ImageDraw.Draw(img)
    everything = zip(wordlist, font_sizes, positions, orientations)
    for wordlist, font_size, position, orientation in everything:
        font = ImageFont.truetype(font_path, font_size)
         #transpose font optionally
        transposed_font = ImageFont.TransposedFont(font, orientation=orientation)
        draw.setfont(transposed_font)
        draw.text((position[1], position[0]), wordlist, #fill = "red")
                   fill="hsl(%d" % random.randint(0, 50) + ", 80%, 50%)")
    #img.show()
    try:
        img.save(fname)
        print('save successful')
    except:
        print('save failed')
    return img


if __name__ == "__main__":

    x=wordlist
    co=vallist
    #from wordcloud import make_wordcloud
    make_wordcloud(x,co,'wordy.jpg')

C:\Users\Suchithra Bala\Anaconda3\lib\site-packages\PIL\ImageDraw.py:99: UserWarning: setfont() is deprecated. Please set the attribute directly instead.
  "Please set the attribute directly instead.")


save successful


## Sentiment Analysis

In [4]:
#sentiment graph creation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt

values = pd.read_csv('Semantic_Orientation.csv')
valuess=values.Value
wordss=values.Word

i=0
vallist=[]
neg=0
pos=0
for words, values in zip(wordss, valuess):
    val=float(valuess[i])
    i=i+1
    if val<0:
        neg=neg+val
    else:
        pos=pos+val
neg=-neg
print(pos)
print(neg)

21128.71954367301
8429.512578909043


In [ ]:
objects = ('Positive', 'Negative')
y_pos = np.arange(len(objects))
performance=[]
performance.append(pos)
performance.append(neg)
 
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Sentiment Scale')
plt.xlabel('Sentiment Class')
plt.title('Sentiment Analysis on Tweets')
 
plt.show()

## Sentiment Classifier

In [82]:
import pandas as pd
import nltk
import re
import string

# read in training data for NLP, this training data is downloaded from kaggle.com for tweet sentiment analysis
nlp_train = pd.read_table('training.txt')

# split train words
train_data = []


def extract_train_word(x, train_data):
    words = [e.lower() for e in nltk.tokenize.word_tokenize(x['Tweet']) 
             if ((len(e) >= 3) and (e not in stop) and (not e.startswith(('#','@'))))]
    sentiment = x['Sentiment']
    train_data.append((words, sentiment))
    
result = nlp_train.apply(lambda x: extract_train_word(x, train_data), axis=1)

# load words pool from Bing Liu
pos_pool = pd.read_table('positive-words.txt')
neg_pool = pd.read_table('negative-words.txt')
pos_pool.columns = ['word']
neg_pool.columns = ['word']
word_pool = pos_pool['word'].tolist() + neg_pool['word'].tolist()

# feature extraction with words pool
def feature_extract(tweets):
    global word_pool
    features = {}
    dry_tweet = set(tweets)
    for word in word_pool:
        features['Contain (%s)'%word] = (word in dry_tweet)
    return features

# generate training set
train_set = nltk.classify.apply_features(feature_extract, train_data)

# train classifier
classifier = nltk.NaiveBayesClassifier.train(train_set)



/home/sylor/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:15: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [93]:
# function to extract words inside the tweet
def extract_word(tweet):
    tweet = re.sub(r"http\S+", "", tweet)
    words = [e.lower() for e in nltk.tokenize.word_tokenize(tweet) 
             if ((len(e) >= 3) and (e not in stop) and (not e.startswith(('#','@'))))]
    return words


# test classify
result_board = ['negative', 'positive']

test_tweet = 'driveless car is unsafe'
res = classifier.classify(feature_extract(extract_word(test_tweet)))
print 'The tweet \'%s\' is %s' % (test_tweet, result_board[res])


The tweet 'driveless car is unsafe' is negative


*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu) *and* the TA Wen Liu (wliu3@wpi.edu).
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

** Totoal Points: 120 **


---------------------------------------------------------------------------
** Notebook:  **
    Points: 80


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 
  
    -----------------------------------
    Qestion 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API.  Come up with a business question and describe how Twitter data can help you answer that question.




---------------------------------------------------------------------------
** Report: communicate the results**
    Points: 20

(1) What data you collected?
    Points: 5 

(2) Why this topic is interesting or important to you? (Motivations)
    Points: 5 

(3) How did you analyse the data?
    Points: 5 

(4) What did you find in the data?
(please include figures or tables in the report, but no source code)
    Points: 5 



---------------------------------------------------------------------------
** Slides (for 10 minutes of presentation): Story-telling **
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

